### LlamaIndexの前準備

In [1]:
# パッケージのインストール
!pip install llama-index==0.10.39
!pip install llama-index-llms-gemini
!pip install llama-index-embeddings-huggingface

  Using cached llama_index_core-0.10.68.post1-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.10.68.post1-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.22
    Uninstalling llama-index-core-0.11.22:
      Successfully uninstalled llama-index-core-0.11.22
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-huggingface 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-readers-youtube-transcript 0.2.0 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-postprocessor-flag-embedding-reranker 0.2.0 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llam

In [2]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
from tqdm.notebook import tqdm

load_dotenv()
GOOGLE_API_KEY=os.environ.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
import logging
import sys

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [4]:
from llama_index.core import Settings
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# LLMの準備
Settings.llm = Gemini(
    model_name="models/gemini-1.5-flash",
    safety_settings={
        "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
        "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
        "HARM_CATEGORY_SEXUALLY_EXPLICIT" : "BLOCK_NONE",
        "HARM_CATEGORY_DANGEROUS_CONTENT" : "BLOCK_NONE"
    }
)

# 埋め込みモデルの準備
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3"
)

INFO:datasets:PyTorch version 2.5.1 available.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/modules.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/README.md HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/modules.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/sentence_bert_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggi

### Faissの使用手順

In [5]:
# パッケージのインストール
!pip install llama-index-vector-stores-faiss
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.4 MB/s eta 0:00:00


In [6]:
from llama_index.core import SimpleDirectoryReader

# ドキュメントの読み込み (dataフォルダにドキュメントを配置しておきます)
documents = SimpleDirectoryReader("data").load_data()

DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 7
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin1.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin2.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin3.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin4.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin5.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin6.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin7.txt


In [7]:
import faiss

# Faissのインデックスの準備
faiss_index = faiss.IndexFlatL2(1024)

DEBUG:faiss.loader:Environment variable FAISS_OPT_LEVEL is not set, so let's pick the instruction set according to the current CPU
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


In [8]:
from llama_index.core import (VectorStoreIndex, StorageContext)
from llama_index.vector_stores.faiss import FaissVectorStore

# インデックスの準備
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context
)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めな...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第3章：情報収集
ネオン街の裏社会に潜伏するアズキとオオカミ。彼らは帝国の秘密情報を収集するた...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第4章：裏切り
ある日、オオカミの裏切りが発覚する。彼は帝国に情報を売っていたのだ。
アズキは...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第5章：決戦
ついに、アズキとオオカミは帝国の本拠地へ潜入する。そこで、彼らは帝国の恐るべき計...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第6章：希望の光
激しい戦いの末、アズキとオオカミはシステムを破壊することに成功する。帝国は崩...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第7章：新たな旅
街に平和が訪れた数年後、アズキは新たな旅に出る。
彼女は世界中を旅し、他の抑...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
VectorStoreIndex()

In [9]:
# クエリエンジンの準備
query_engine = index.as_query_engine()

In [10]:
# 質問応答
response = query_engine.query("アズキの年齢は？")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node 0] [Similarity score:             0.910998] 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地裏を疾走していた。彼女は16歳、ハッカー集団「紅ずきん」の一員だ。
アズキのターゲットは、企業連合「...
> [Node 1] [Similarity score:             0.978581] 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めなかった。脱出する方法を探るアズキの前に、謎めいた青年が現れる。
青年はハッカー「オオカミ」と呼ばれ、...
アズキは16歳です。 



### Pineconeを使った質問応答

In [ ]:
# パッケージのインストール
!pip install llama-index-vector-stores-pinecone
!pip install pinecone-client

In [ ]:
from llama_index.core import SimpleDirectoryReader

# ドキュメントの読み込み (dataフォルダにドキュメントを配置しておきます)
documents = SimpleDirectoryReader("data").load_data()

In [ ]:
from google.colab import userdata
import os

# 環境変数の準備 (左端の鍵アイコンでPINECONE_API_KEYを設定)
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")

In [ ]:
from pinecone import Pinecone

# Pineconeオブジェクトの準備
pc = Pinecone(
    api_key=os.environ["PINECONE_API_KEY"]
)

In [ ]:
# 既存のPineconeのインデックスの取得
indexes = pc.list_indexes()
print(indexes)

In [ ]:
from pinecone import PodSpec

# Pineconeのインデックスの作成
if "quickstart-index" not in indexes:
    pc.create_index(
        "quickstart-index",
        dimension=1024,
        metric="dotproduct",
        spec=PodSpec(
            environment="us-west1-gcp",
            pod_type="p1.x1",
            pods=1
        )
    )

In [ ]:
# Pineconeのインデックスの準備
pc_index = pc.Index("quickstart-index")

In [ ]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex

# インデックスの準備
vector_store = PineconeVectorStore(
    pinecone_index=pc_index,
    text_key="content"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context
)

In [ ]:
# クエリエンジンの準備
query_engine = index.as_query_engine()

In [ ]:
# 質問応答
response = query_engine.query("アズキの年齢は？")
print(response)